In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader  # 주어진 dataset과 sampler를 감싸서 iterable한 객체로 반환하는 역할
from torchvision import datasets  # 라이브러리에서 주어지는 데이터셋을 불러오는 용도
from torchvision.transforms import ToTensor  # PIL Image나 Numpy를 FloatTensor로 변환하고 이미지의 픽셀의 크기를 [0, 1] 범위로 비례하여 조정

In [19]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [20]:
batch_size = 64  # 한번에 학습할 개수, 전체 데이터를 한번에 학습하기엔 양이 많으므로 설정해주는 것

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")  # N: batch_size와 동일, C: Channel, 3인 경우 컬러, 1인 경우 흑백, H: Height, W: Width
    # 일반적으로 이미지는 H, W, C 순서로 나타낸다, 28*28*1
    
    # flatten = nn.Flatten()
    # flat_image = flatten(X)
    # print(flat_image.shape)
    print(f"Shape of y: {y.shape} {y.dtype}")  # batch_size가 64로 설정되었으므로 64의 길이를 가지는 배열로 만들었다고 생각하면 될 듯 싶다
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


#### Create Models

In [21]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'Using {device} device')

Using cpu device


In [22]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()  # Linear Layer를 사용하기 위해 Flatten해준다, batch_size * 784 형태의 배열이 만들어진다
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):  # model(x)로 호출하면 forward 함수가 호출되어 값을 반환한다는데, 모델을 학습한 후 예측할 때 쓰이는 것인가 싶음, 정확히 이해는 안된다
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### Optimizing the Model Parameters

In [23]:
loss_fn = nn.CrossEntropyLoss()  # loss function, 손실 함수라고 불리며 예측값과 실제값 간의 오차를 의미
# 종류로는 MSE(Mean Squard Error), RMSE(Root Mean Squared Error), Binary Crossentropy(레이블의 개수가 2개인 경우 사용)
# https://velog.io/@rcchun/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EC%86%90%EC%8B%A4%ED%95%A8%EC%88%98%EC%9D%98-%EC%A2%85%EB%A5%98#categorical-crossentropy
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)  # 손실 함수의 최솟값을 찾아나가는 방법, 과정을 의미
# https://blog.kakaocdn.net/dn/bQ934t/btqASyVqeeD/ozNDSKWvAbxiJb7VtgLkSk/img.png

In [24]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)  # 예측값
        loss = loss_fn(pred, y)  # 오차
        
        # Backpropagation, 역전파라고 부름
        # https://evan-moon.github.io/2018/07/19/deep-learning-backpropagation/
        
        loss.backward()  # prediction loss를 역전파
        optimizer.step()  # 역전파 단계에서 수집된 변화도로 매개변수를 조정
        optimizer.zero_grad()  # 매개변수의 변화도는 기본적으로 더해지기 때문에 중복 계산을 막기 위해서 0으로 설정
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)  # len(X)는 batch_size와 같다
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [25]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():  # gradient 계산을 비활성화하는 역할
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [26]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t + 1}\n-----------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------------------
loss: 2.297910 [   64/60000]


loss: 2.285151 [ 6464/60000]
loss: 2.261744 [12864/60000]
loss: 2.254677 [19264/60000]
loss: 2.241296 [25664/60000]
loss: 2.210982 [32064/60000]
loss: 2.222811 [38464/60000]
loss: 2.183017 [44864/60000]
loss: 2.174458 [51264/60000]
loss: 2.143850 [57664/60000]
Test Error: 
 Accuracy: 44.7%, Avg loss: 2.135218

Epoch 2
-----------------------------------
loss: 2.148204 [   64/60000]
loss: 2.138039 [ 6464/60000]
loss: 2.071001 [12864/60000]
loss: 2.084370 [19264/60000]
loss: 2.038544 [25664/60000]
loss: 1.978342 [32064/60000]
loss: 2.006993 [38464/60000]
loss: 1.917298 [44864/60000]
loss: 1.924068 [51264/60000]
loss: 1.852897 [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.843811

Epoch 3
-----------------------------------
loss: 1.882779 [   64/60000]
loss: 1.855519 [ 6464/60000]
loss: 1.723176 [12864/60000]
loss: 1.763973 [19264/60000]
loss: 1.662400 [25664/60000]
loss: 1.619750 [32064/60000]
loss: 1.643136 [38464/60000]
loss: 1.535693 [44864/60000]
loss: 1.569164 [51264/60000

#### Saving, Loading Model

In [27]:
# torch.save(model.state_dict(), 'model.pth')
# print('Saved PyTorch Model State to model.pth')

In [28]:
# model = NeuralNetwork().to(device)
# model.load_state_dict(torch.load("model.pth"))

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()  # 모듈을 평가 모드로 설정
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
